# Parse Json

In [ ]:
def parseRaw(json_map):
    url = json_map['url']
    content = json_map['html']
    return (url,content)

# 載入原始 RAW Data

In [ ]:
import json
import pprint
pp = pprint.PrettyPrinter(indent=2)
path = "./pixnet.txt"
all_content = sc.textFile(path).map(json.loads).map(parseRaw)

# 利用 LXML Parser 來分析文章結構

In [ ]:
def parseImgSrc(x):
    try:
        urls = list()
        import lxml.html
        from urlparse import urlparse
        node = lxml.html.fromstring(x)
        root =  node.getroottree()
        for src in root.xpath('//img/@src'):
            try :
                host = urlparse(src).netloc
                if '.' not in host : continue
                if host.count('.') == 1 : 
                    pass
                else: 
                    host = host[host.index('.')+1:]
                urls.append('imgsrc_'+host)
            except :
                print "Error Parse At:" , src
            
        for src in root.xpath('//input[@src]/@src'):
            try :
                host = urlparse(src).netloc
                if '.' not in host : continue
                if host.count('.') == 1 : 
                    pass
                else: 
                    host = host[host.index('.')+1:]
                urls.append('imgsrc_'+host)
            except :
                print "Error parseImgSrc At:" , src
        
    except :
        print "Unexpected error:", sys.exc_info()
    return  urls

In [ ]:
all_content.map(lambda x: x[1]).first()[:100]

# 取出 Image Src 的列表

In [ ]:
image_list = all_content.map(lambda x :parseImgSrc(x[1]))
pp.pprint(image_list.first()[:10])

# 統計 Image Src 的列表

In [ ]:
img_src_count = all_content.map(
    lambda x :parseImgSrc(x[1])).flatMap(
    lambda x: x).countByValue()
for i in img_src_count:
    print i , ':' , img_src_count[i]

#<span style="color: blue">請使用　reduceByKey , sortBy 來計算出 img src 排行榜</span>
請參照以下文件
[http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD]

In [ ]:
from operator import add
all_content.map(
    lambda x :parseImgSrc(x[1])).???

#正確的排行如下：
<code> 
[('imgsrc_pixfs.net', 219),
 ('imgsrc_agoda.net', 103),
 ('imgsrc_static.flickr.com', 53),
 ('imgsrc_staticflickr.com', 28),
 ('imgsrc_pimg.tw', 19),
 ('imgsrc_facebook.com', 12),
 ('imgsrc_sitebro.com', 10),
 ('imgsrc_linkwithin.com', 5),
 ('imgsrc_cloudfront.net', 5),
 ('imgsrc_prchecker.info', 5),
 ('imgsrc_visit-japan.jp', 5),
 ('imgsrc_yimg.com', 2),
 ('imgsrc_zenfs.com', 2),
 ('imgsrc_googleusercontent.com', 1)]
</code>